# Measuring Term Co-occurrence

In the Pubmed corpus, certain pairs of MeSH terms have a significantly higher probability of being applied to a single Pubmed document (citation) than would be expected based on their individual probabilities of being applied to a document. I am hoping that I might be able to use term co-occurrence log-likelihood ratios in order to help inform decision making in my models.

## Setup

Building co-occurrence matrices can be a very memory-intensive task - especially with the number of MeSH terms totalling 29,350 and the current Pubmed corpus containing ~29.1 documents. Determining term co-occurrence in a single step by multiplying the corpus's term-document matrix by its transposed matrix would require tens of terabytes of RAM, so I broke the problem down into more digestable pieces. I used a generator to generate term-document matrices of a fixed size one-at-a-time in order to avoid storing data in memory. I then simply added the co-occurrence matrices with an array in order to keep track of the totals. In order to make things a little easier initially, I decided to first look at co-occurrences only for the top 7,221 most common MeSH terms.

### Imports and Function Definitions

In [ ]:
#!/usr/bin/env python3
import os
import re
import time
import math
import argparse
import logging
import traceback
from multiprocessing import Process, Queue

import numpy as np
from tqdm import tqdm

def count_doc_terms(doc_list, term_subset, logger):
    doc_terms = {}
    doc_pmid = ""
    term_ids = []

    term_set = set(term_subset)

    # Compile regexes
    pm_article_start = re.compile(r"\s*<PubmedArticle>")
    pm_article_stop = re.compile(r"\s*</PubmedArticle>")
    pmid = re.compile(r"\s*<PMID.*>(\d*)</PMID>")
    mesh_list_start = re.compile(r"\s*<MeshHeadingList>")
    mesh_list_stop = re.compile(r"\s*</MeshHeadingList>")
    mesh_term_id = re.compile(r'\s*<DescriptorName UI="(D\d+)".*>')

    logger.info("Starting doc/term counting")
    for doc in tqdm(doc_list):
        try:
            with open(f"./pubmed_bulk/{doc}", "r") as handle:
                start_doc_count = len(doc_terms.keys())
                start_time = time.perf_counter()

                line = handle.readline()
                while line:
                    if pm_article_start.search(line):
                        if doc_pmid:
                            doc_terms[doc_pmid] = term_ids
                            doc_pmid = ""
                            term_ids = []
                        while not pm_article_stop.search(line):
                            if not doc_pmid and pmid.search(line):
                                doc_pmid = pmid.search(line).group(1)
                            if mesh_list_start.search(line):
                                while not mesh_list_stop.search(line):
                                    mesh_match = mesh_term_id.search(line)
                                    if mesh_match and mesh_match.group(1) in term_set:
                                        term_ids.append(mesh_match.group(1))
                                    line = handle.readline()
                            line = handle.readline()
                    line = handle.readline()
                doc_terms[doc_pmid] = term_ids

                # Get count for log
                docs_counted = len(doc_terms.keys()) - start_doc_count
                # Get elapsed time and truncate for log
                elapsed_time = int((time.perf_counter() - start_time) * 10) / 10.0
                logger.info(f"{doc} parsing completed - terms extracted for {docs_counted} documents in {elapsed_time} seconds")
                
        except Exception as e:
            trace = traceback.format_exc()
            logger.error(repr(e))
            logger.critical(trace)

    logger.info("Stopping doc/term counting")

    with open("./data/pm_bulk_doc_term_counts.csv", "w") as out:
        for doc in doc_terms:
            out.write("".join([doc, ","]))
            out.write(",".join(doc_terms[doc]))
            out.write("\n")

def td_matrix_gen(file_path, term_subset, docs_per_matrix):
    with open("./data/pm_bulk_doc_term_counts.csv", "r") as handle:
        td_matrix = []
        for line in handle:
            if len(td_matrix) > docs_per_matrix:
                yield td_matrix
                td_matrix = []
            line = line.strip("\n").split(",")
            terms = line[1:]
            row = []
            for uid in term_subset:
                if uid in terms:
                    row.append(1)
                else:
                    row.append(0)
            td_matrix.append(row)

# A function for multiprocessing
def matrix_builder(work_queue, add_queue, id_num):
    # Set up logging - I do actually want a logger for each worker
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    handler = logging.FileHandler(f"./logs/term_co-occurrence_worker{id_num}.log")
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

    try:
        while True:
            matrix = work_queue.get()
            if matrix is None:
                break
            td_matrix = np.array(matrix)
            co_matrix = np.dot(td_matrix.transpose(), td_matrix)
            add_queue.put(co_matrix)

    except Exception as e:
        trace = traceback.format_exc()
        logger.error(repr(e))
        logger.critical(trace)

# A function for multiprocessing, pulls from the queue and writes
def matrix_adder(add_queue, completed_queue, dim, docs_per_matrix, num, logger):
    # Log counts and rates after every n matrices
    log_interval = 800
    total_processed = 0
    co_matrix = np.zeros((dim, dim))
    start_time = time.perf_counter()
    while True:
        if total_processed and total_processed % log_interval == 0:
            elapsed_time = int((time.perf_counter() - start_time) * 10) / 10.0
            time_per_it = elapsed_time / (docs_per_matrix * log_interval)
            logger.info(f"Adder {num}: {total_processed * docs_per_matrix} docs added to matrix - last batch of "
                        f"{docs_per_matrix * log_interval} at a rate of {time_per_it} sec/it")
            start_time = time.perf_counter()

        matrix_to_add = add_queue.get()
        if matrix_to_add is None:
            completed_queue.put(co_matrix)
            break
        co_matrix = co_matrix + matrix_to_add
        total_processed += 1

## Main Processing

Please do not try to run this code in this notebook - see the [original code](https://github.com/wigasper/FUSE/blob/master/term_co-occurrence.py) which I am always in the process of making more portable and usable. There is a lot of bloat here related to the multiprocessing architecture, but as I implemented it this way from the start, and never tried a single-threaded approach, I am not going to rewrite the code for single-threaded logic for the sake of this notebook.

In [ ]:
def main():
    # Get command line args
    parser = argparse.ArgumentParser()
    parser.add_argument("-rc", "--recount", help="recount terms for each doc", type=str)
    parser.add_argument("-n", "--num_docs", help="number of docs to build co-occurrence matrix with", type=int)
    args = parser.parse_args()

    # Set up logging
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    handler = logging.FileHandler("./logs/term_co-occurrence.log")
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

    # Load term subset to count for
    term_subset = []
    with open("./data/subset_terms_list", "r") as handle:
        for line in handle:
            term_subset.append(line.strip("\n"))

    if args.recount:
        docs = os.listdir("./pubmed_bulk")
        count_doc_terms(docs, term_subset, logger)

    # This value was determined in testing but is kind of arbitrary
    # Maybe need to figure out a better way to get this
    docs_per_matrix = 34

    matrix_gen = td_matrix_gen("./data/pm_bulk_doc_term_counts.csv", term_subset, docs_per_matrix)

    # Set up multiprocessing
    num_builders = 5
    num_adders = 2
    add_queue = Queue(maxsize=5)
    build_queue = Queue(maxsize=num_builders)
    completed_queue = Queue(maxsize=num_adders + 1)

    logger.info(f"Starting workers on {args.num_docs} docs")

    adders = [Process(target=matrix_adder, args=(add_queue, completed_queue, len(term_subset), 
                    docs_per_matrix, num, logger)) for num in range(num_adders)]
    
    for adder in adders:
        #adder.daemon = True
        adder.start()

    builders = [Process(target=matrix_builder, args=(build_queue, add_queue, num)) for num in range(num_builders)]

    for builder in builders:
        builder.start()

    if args.num_docs:
        limit = args.num_docs / docs_per_matrix
    else:
        limit = math.inf
    
    doc_count = 0
    for matrix in matrix_gen:
        if doc_count < limit:
            build_queue.put(matrix)
            doc_count += 1
        else:
            break
    
    while True:
        if build_queue.empty():
            for _ in range(num_builders):
                build_queue.put(None)
            break

    for builder in builders:
        builder.join()

    for adder in adders:
        add_queue.put(None)

    if add_queue.empty():
        for adder in adders:
            adder.join()

    co_matrices = []
    for _ in range(num_adders):
        co_matrices.append(completed_queue.get())

    co_matrix = sum(co_matrices)
    
    np.save("./data/co-occurrence-matrix", co_matrix)

    # Compute probabilities to compare against
    term_counts = {}

    with open("./data/mesh_data.tab", "r") as handle:
        for line in handle:
            line = line.strip("\n").split("\t")
            term_counts[line[0]] = 0

    with open("./data/pm_bulk_doc_term_counts.csv", "r") as handle:
        for _ in range(doc_count):
            line = handle.readline()
            line = line.strip("\n").split(",")
            terms = line[1:]
            terms = [term for term in terms if term]
            for term in terms:
                term_counts[term] += 1
    
    # Get probability of each term for the document set
    total_terms = sum(term_counts.values())

    for term in term_counts:
        term_counts[term] = term_counts[term] / total_terms

    # Create the expected co-occurrence probability matrix
    expected = np.zeros((len(term_subset), len(term_subset)))

    for row in range(expected.shape[0]):
        for col in range(expected.shape[1]):
            expected[row, col] = term_counts[term_subset[row]] * term_counts[term_subset[col]]

    # Fill 0s with np.NaN to avoid zero division errors later
    expected[expected == 0] = np.NaN

    # Get the total number of co-occurrences
    total_cooccurrs = 0
    for row in range(co_matrix.shape[0]):
        for col in range(co_matrix.shape[1]):
            if row != col:
                total_cooccurrs += co_matrix[row, col]
    total_cooccurs = total_cooccurrs / 2

    # Create an array just consisting of the total number of co-occurrences in the corpus
    temp_total_array = np.full((len(co_matrix), len(co_matrix)), total_cooccurs)

    # Divide each number of co-occurrences by the the total to get P(co-occurrence)
    co_matrix = np.divide(co_matrix, temp_total_array)

    # Divide P(co-occurrence) by expected P(co-occurrence) to get the likelihood ratio
    likelihood_ratios = np.divide(co_matrix, expected)

    # Set 0s to NaN to avoid taking log(0)
    likelihood_ratios[likelihood_ratios == 0] = np.NaN
    
    # Take the log of the array to get the log-likelihood ratio
    log_ratios = np.log(likelihood_ratios)
    
    with open("./data/term_co-occ_log_likelihoods.csv", "w") as out:
        for row in range(log_ratios.shape[0]):
            for col in range(row + 1, log_ratios.shape[1]):
                if not np.isnan(log_ratios[row,col]):
                    out.write(",".join([term_subset[row], term_subset[col], str(log_ratios[row,col])]))

## Results

These are some of the top scoring pairs of terms in terms of their log-likelihood ratios - that is, the number of co-occurrences of these pairs of terms was significantly higher than would be expected given their probabilities of occurring individually. It's easy to see how most of the the co-occurrence of many of the top scoring pairs is overrepresented - they are relatively specific terms, so they are unlikely to be in a large amount of documents, and they tend to be closely related. The second most "enriched" term pair, "cerebellar cortex" and "population characteristics", is quite interesting though.

The MeSH UID: name dictionary was created by with data obtained by [parsing MeSH](https://github.com/wigasper/FUSE/blob/master/data-aggregation/parse_mesh.py).

This data may be affected by using a subset of the Pubmed corpus - these results were generated using 1.9 million documents out of 29 million. In order to have a high log-likelihood ratio, a pair of terms is much more likely to be comprised of very rare terms (because their expected co-occurrance probability is then extremely small). Because of this, it is quite possible that inclusion of a larger portion of the corpus will significantly affect the log-likelihood ratios. 

In [10]:
# Load in the log likelihood ratios for term pairs
log_likelihood_ratios = []
with open("./data/term_co-occ_log_likelihoods.csv", "r") as handle:
    for line in handle:
        line = line.strip("\n").split(",")
        line[2] = float(line[2])
        log_likelihood_ratios.append(line)

# Load in MeSH data to get term names dict
term_names = {}

with open("./data/mesh_data.tab", "r") as handle:
    for line in handle:
        line = line.strip("\n").split("\t")
        term_names[line[0]] = line[1]
            
sorted_ratios = sorted(log_likelihood_ratios, key=lambda l:l[2], reverse=True)

for ratio in sorted_ratios[0:20]:
    print("".join([term_names[ratio[0]], " & ", term_names[ratio[1]], " - log-likelihood ratio: ", str(ratio[2])]))

History, Early Modern 1451-1600 & History, Modern 1601- - log-likelihood ratio: 9.388012728071354
Cerebellar Cortex & Population Characteristics - log-likelihood ratio: 9.166820396888406
Sulfamethoxazole & Trimethoprim - log-likelihood ratio: 9.160013526464441
Endothelial Growth Factors & Vascular Endothelial Growth Factors - log-likelihood ratio: 8.88587974684798
Sevoflurane & Methyl Ethers - log-likelihood ratio: 8.79957650160044
Atorvastatin & Heptanoic Acids - log-likelihood ratio: 8.794538707570482
Polylactic Acid-Polyglycolic Acid Copolymer & Polyglycolic Acid - log-likelihood ratio: 8.730583630113488
Orthodontic Appliances & Orthodontics, Corrective - log-likelihood ratio: 8.68050817100347
Hexuronic Acids & Glucuronic Acid - log-likelihood ratio: 8.578496508591751
Glycogen Synthase Kinase 3 beta & Glycogen Synthase Kinase 3 - log-likelihood ratio: 8.550509265276437
Rotavirus Infections & Rotavirus - log-likelihood ratio: 8.54157487439918
Irinotecan & Camptothecin - log-likelihoo

Likewise, the lowest scoring pairs are also interesting. These are pairs of terms that would be expected to co-occur at a higher rate than they did in actuality. Unlike the highest scoring pairs, these pairs are generally going to be comprised of frequent terms (which naturally have a higher expected co-occurrence probability) that do not make sense as pairs in the context of research.

In [11]:
sorted_ratios_rev = sorted(log_likelihood_ratios, key=lambda l:l[2])

for ratio in sorted_ratios_rev[0:20]:
    print("".join([term_names[ratio[0]], " & ", term_names[ratio[1]], " - log-likelihood ratio: ", str(ratio[2])]))

Animals & Managed Care Programs - log-likelihood ratio: -6.139310713749786
Animals & Medicare - log-likelihood ratio: -6.022275494165508
Middle Aged & Crystallography, X-Ray - log-likelihood ratio: -5.942771983493556
United States & Cell Line, Tumor - log-likelihood ratio: -5.703295107437918
Animals & Nursing Staff, Hospital - log-likelihood ratio: -5.637691762109699
Rats, Inbred Strains & United States - log-likelihood ratio: -5.487749788700135
Middle Aged & Gene Expression Regulation, Plant - log-likelihood ratio: -5.4489061847709745
Animals & Hospitals, Psychiatric - log-likelihood ratio: -5.394812966673756
Adult & Crystallography, X-Ray - log-likelihood ratio: -5.390534872050586
Animals & Medical Records Systems, Computerized - log-likelihood ratio: -5.362214038191328
Kinetics & Surveys and Questionnaires - log-likelihood ratio: -5.342553058886607
Drosophila melanogaster & Middle Aged - log-likelihood ratio: -5.322452029751443
Attitude of Health Personnel & Rats - log-likelihood ra